In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
% pip install wandb -q

In [ ]:
% pip install pdbpp

In [3]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
# Turning off tensorflow warning message (not recommended)

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

In [4]:
import numpy as np
import tensorflow as tf
import tensorflow.keras.layers as layers
import tensorflow.keras.backend as kb
from tensorflow.keras.models import save_model
from tensorflow.keras.callbacks import Callback
import random
import pdb
import wandb
from os.path import join, isfile
from os import listdir

In [ ]:
print(tf.__version__)

1.15.2


In [6]:
wandb.login()

wandb: Currently logged in as: awesomericky (use `wandb login --relogin` to force relogin)


True

In [18]:
device_name = tf.test.gpu_device_name()
print('Found GPU at {}'.format(device_name))

Found GPU at /device:GPU:0


In [19]:
"""
utils.py
"""

def read_script_files(script_dir_path):
    script_files = [f for f in listdir(script_dir_path) if isfile(join(script_dir_path, f))]
    script_files.sort()
    return script_files

def data_shuffle(relative_data_directory_path):
    data_files = read_script_files(relative_data_directory_path)
    random.shuffle(data_files)
    return data_files

In [20]:
"""
processed_data_loader.py
"""

def load_single_npz_data(relative_data_directory_path, file_name):
    file_path = join(relative_data_directory_path, file_name)
    data = np.load(file=file_path)
    return data

In [49]:
"""
TCN_and_decoder.py
"""

class ResidualBlock(tf.keras.Model):
    def __init__(self, dilation_rate, num_filters, kernel_size, padding, 
                        dropout_rate, seed, training_state):
        super(ResidualBlock, self).__init__()
        init = tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.1, seed=seed)
        layers = tf.keras.layers
        assert padding in ['causal', 'same']

        self.training_state = training_state

        # Block1
        self.conv1 = layers.Conv1D(filters=num_filters, kernel_size=kernel_size, data_format='channels_last',
                                    dilation_rate=dilation_rate, padding=padding, kernel_initializer=init)
        self.batch1 = layers.BatchNormalization(axis=1, trainable=True)
        self.ac1 = layers.LeakyReLU(alpha=0.2)
        self.drop1 = layers.Dropout(rate=dropout_rate)

        # Block2
        self.conv2 = layers.Conv1D(filters=num_filters, kernel_size=kernel_size, data_format='channels_last',
                                    dilation_rate=dilation_rate, padding=padding, kernel_initializer=init)
        self.batch2 = layers.BatchNormalization(axis=1, trainable=True)
        self.ac2 = layers.LeakyReLU(alpha=0.2)
        self.drop2 = layers.Dropout(rate=dropout_rate)

        self.downsample = layers.Conv1D(filters=num_filters, kernel_size=1,
                                        padding='same', kernel_initializer=init)
        self.ac3 = layers.LeakyReLU(alpha=0.2)
    
    def call(self, x):
        # Block1
        prev_x = x
        x = self.conv1(x)
        x = self.batch1(x)
        x = self.ac1(x)
        x = self.drop1(x) if self.training_state else x

        # Block2
        x = self.conv2(x)
        x = self.batch2(x)
        x = self.ac2(x)
        x = self.drop2(x) if self.training_state else x

        # Match dimention
        if prev_x.shape[-1] != x.shape[-1]:
            prev_x = self.downsample(prev_x)
        assert prev_x.shape[1:] == x.shape[1:]

        # skip connection
        return self.ac3(prev_x + x)

class TemporalBlock(tf.keras.Model):
    def __init__(self, num_channels, kernel_size, dropout_rate, seed, training_state):
        # num_channels is a list contains hidden channel numbers of Conv1D
        # len(num_channels) is number of convolutional layers in one Temporal Block
        super(TemporalBlock, self).__init__()
        assert isinstance(num_channels, list)

        self.num_levels = len(num_channels)
        self.resi_blocks = [0]*self.num_levels
        for i in range(self.num_levels):
            dilation_rate = 2**i
            self.resi_blocks[i] = ResidualBlock(dilation_rate, num_channels[i], kernel_size, padding='causal',
                            dropout_rate=dropout_rate, seed=seed, training_state=training_state)
    
    def call(self, x):
        for i in range(self.num_levels):
            x = self.resi_blocks[i](x)
        return x

class TempConvnet(tf.keras.Model):
    def __init__(self, num_stacks, num_channels, kernel_size, dropout_rate, return_type, seed, training_state):
        # num_stacks number of Temporal Blocks in Temporal convolutional network
        super(TempConvnet, self).__init__()
        assert isinstance(num_stacks, int)
        assert isinstance(num_channels, list)
        assert return_type in ['whole', 'end']

        self.num_stacks = num_stacks
        self.temp_blocks = [0]*self.num_stacks
        self.return_type = return_type
        for i in range(num_stacks):
            self.temp_blocks[i] = TemporalBlock(num_channels, kernel_size=kernel_size, dropout_rate=dropout_rate, seed=seed, training_state=training_state)
    
    def call(self, x):
        for i in range(self.num_stacks):
            x = self.temp_blocks[i](x)
        
        if self.return_type == 'whole':
            return x
        elif self.return_type == 'end':
            return x[:, -1, :]

class TempConvnet_Decoder(tf.keras.Model):
    def __init__(self, decoder_type, num_channels, kernel_size, padding, dropout_rate, seed, training_state):
        super(TempConvnet_Decoder, self).__init__()
        assert isinstance(num_channels, int)
        assert isinstance(kernel_size, list)
        assert padding in ['causal', 'same']
        assert decoder_type in ['linguistic', 'acoustic']

        init = tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.1, seed=seed)
        layers = tf.keras.layers

        self.training_state = training_state

        # Block1
        self.upsample_block1 = layers.UpSampling1D(size=3)
        self.conv_block1 = layers.Conv1D(filters=num_channels, kernel_size=kernel_size[0], data_format='channels_last',
                                            padding=padding, kernel_initializer=init)
        self.batchnorm_block1 = layers.BatchNormalization(axis=-1, trainable=True)
        self.ac_block1 = layers.ReLU() if decoder_type == 'linguistic' else layers.LeakyReLU(alpha=0.2)
        self.drop_block1 = layers.Dropout(rate=dropout_rate)

        # Block2
        self.upsample_block2 = layers.UpSampling1D(size=3)
        self.conv_block2 = layers.Conv1D(filters=num_channels, kernel_size=kernel_size[0], data_format='channels_last',
                                            padding=padding, kernel_initializer=init)
        self.batchnorm_block2 = layers.BatchNormalization(axis=-1, trainable=True)
        self.ac_block2 = layers.ReLU() if decoder_type == 'linguistic' else layers.LeakyReLU(alpha=0.2)
        self.drop_block2 = layers.Dropout(rate=dropout_rate)

        # Block3
        self.upsample_block3 = layers.UpSampling1D(size=2)
        self.conv_block3 = layers.Conv1D(filters=num_channels, kernel_size=kernel_size[0], data_format='channels_last',
                                            padding=padding, kernel_initializer=init)
        self.batchnorm_block3 = layers.BatchNormalization(axis=-1, trainable=True)
        self.ac_block3 = layers.ReLU() if decoder_type == 'linguistic' else layers.LeakyReLU(alpha=0.2)
        self.drop_block3 = layers.Dropout(rate=dropout_rate)

        # Block4
        self.conv_block4 = layers.Conv1D(filters=num_channels, kernel_size=2, data_format='channels_last',
                                            padding='valid', kernel_initializer=init)
        self.batchnorm_block4 = layers.BatchNormalization(axis=-1, trainable=True)
        self.ac_block4 = layers.ReLU() if decoder_type == 'linguistic' else layers.LeakyReLU(alpha=0.2)
        self.drop_block4 = layers.Dropout(rate=dropout_rate)

        # Block5
        self.upsample_block5 = layers.UpSampling1D(size=2)
        self.conv_block5 = layers.Conv1D(filters=num_channels, kernel_size=kernel_size[0], data_format='channels_last',
                                            padding=padding, kernel_initializer=init)
        self.batchnorm_block5 = layers.BatchNormalization(axis=-1, trainable=True)
        self.ac_block5 = layers.ReLU() if decoder_type == 'linguistic' else layers.LeakyReLU(alpha=0.2)
        self.drop_block5 = layers.Dropout(rate=dropout_rate)

        # Block6
        self.upsample_block6 = layers.UpSampling1D(size=3)
        self.conv_block6 = layers.Conv1D(filters=num_channels, kernel_size=kernel_size[0], data_format='channels_last',
                                            padding=padding, kernel_initializer=init)
        self.batchnorm_block6 = layers.BatchNormalization(axis=-1, trainable=True)
        self.ac_block6 = layers.ReLU() if decoder_type == 'linguistic' else layers.LeakyReLU(alpha=0.2)
        self.drop_block6 = layers.Dropout(rate=dropout_rate)

        # Block7
        self.conv_block7 = layers.Conv1D(filters=num_channels, kernel_size=2, data_format='channels_last',
                                            padding='valid', kernel_initializer=init)
        self.batchnorm_block7 = layers.BatchNormalization(axis=-1, trainable=True)
        self.ac_block7 = layers.ReLU() if decoder_type == 'linguistic' else layers.LeakyReLU(alpha=0.2)
        self.drop_block7 = layers.Dropout(rate=dropout_rate)

        # Block8
        self.upsample_block8 = layers.UpSampling1D(size=3)
        self.conv_block8 = layers.Conv1D(filters=num_channels, kernel_size=kernel_size[0], data_format='channels_last',
                                            padding=padding, kernel_initializer=init)
        self.batchnorm_block8 = layers.BatchNormalization(axis=-1, trainable=True)
        self.ac_block8 = layers.ReLU() if decoder_type == 'linguistic' else layers.LeakyReLU(alpha=0.2)
        self.drop_block8 = layers.Dropout(rate=dropout_rate)

        # Block9 (final block)
        self.final_conv_block = layers.Conv1D(filters=num_channels, kernel_size=kernel_size[1], data_format='channels_last',
                                                    padding=padding, kernel_initializer=init)
        self.final_ac_block = layers.Softmax(axis=-1) if decoder_type == 'linguistic' else layers.LeakyReLU(alpha=0.2)
        self.final_reshape_block = layers.Permute((2, 1))
    
    def call(self, x):
        x = self.upsample_block1(x)
        x = self.conv_block1(x)
        x = self.batchnorm_block1(x)
        x = self.ac_block1(x)
        x = self.drop_block1(x) if self.training_state else x
        x = self.upsample_block2(x)
        x = self.conv_block2(x)
        x = self.batchnorm_block2(x)
        x = self.ac_block2(x)
        x = self.drop_block2(x) if self.training_state else x
        x = self.upsample_block3(x)
        x = self.conv_block3(x)
        x = self.batchnorm_block3(x)
        x = self.ac_block3(x)
        x = self.drop_block3(x) if self.training_state else x
        x = self.conv_block4(x)
        x = self.batchnorm_block4(x)
        x = self.ac_block4(x)
        x = self.drop_block4(x) if self.training_state else x
        x = self.upsample_block5(x)
        x = self.conv_block5(x)
        x = self.batchnorm_block5(x)
        x = self.ac_block5(x)
        x = self.drop_block5(x) if self.training_state else x
        x = self.upsample_block6(x)
        x = self.conv_block6(x)
        x = self.batchnorm_block6(x)
        x = self.ac_block6(x)
        x = self.drop_block6(x) if self.training_state else x
        x = self.conv_block7(x)
        x = self.batchnorm_block7(x)
        x = self.ac_block7(x)
        x = self.drop_block7(x) if self.training_state else x
        x = self.upsample_block8(x)
        x = self.conv_block8(x)
        x = self.batchnorm_block8(x)
        x = self.ac_block8(x)
        x = self.drop_block8(x) if self.training_state else x
        x = self.final_conv_block(x)
        x = self.final_ac_block(x)
        y = self.final_reshape_block(x)
        return y


########################
### Previous version ###
########################

# class TempConvnet_Decoder(tf.keras.Model):
#     def __init__(self, decoder_type, num_levels, num_channels, kernel_size, padding, upsample_size, dropout_rate, output_shape, seed, training_state):
#         super(TempConvnet_Decoder, self).__init__()
#         assert isinstance(num_channels, int)
#         assert isinstance(kernel_size, list)
#         assert isinstance(upsample_size, dict)
#         assert padding in ['causal', 'same']
#         assert decoder_type in ['linguistic', 'acoustic']
#         assert num_levels == upsample_size[2] + upsample_size[3] + upsample_size[0]

#         init = tf.keras.initializers.RandomNormal(mean=0.0, stddev=1, seed=seed)
#         layers = tf.keras.layers

#         self.training_state = training_state

#         self.size2_upsample_num = upsample_size[2]
#         self.size3_upsample_num = upsample_size[3]
#         self.reshape_conv_num = 1
#         self.num_levels = num_levels
#         self.final_kernel_size = kernel_size[1]
#         self.output_len = output_shape[-1]  ## ex) (num_batch, num_time_steps, dictionary_length)

#         self.upsample_blocks = [0]*(num_levels-1)
#         self.conv_blocks = [0]*(num_levels-1)
#         self.batchnorm_blocks = [0]*(num_levels-1)
#         self.ac_blocks = [0]*(num_levels-1)
#         self.drop_blocks = [0]*(num_levels-1)

#         for i in range(num_levels-self.size3_upsample_num-self.reshape_conv_num):
#             self.upsample_blocks[i] = layers.UpSampling1D(size=2)
#             self.conv_blocks[i] = layers.Conv1D(filters=num_channels, kernel_size=kernel_size[0], data_format='channels_last',
#                                                     padding=padding, kernel_initializer=init)
#             self.batchnorm_blocks[i] = layers.BatchNormalization(axis=-1, trainable=True)
#             self.ac_blocks[i] = layers.ReLU() if decoder_type == 'linguistic' else layers.LeakyReLU(alpha=0.2)
#             self.drop_blocks[i] = layers.Dropout(rate=dropout_rate)
#         for i in range(num_levels-self.size3_upsample_num-self.reshape_conv_num, num_levels-self.reshape_conv_num):
#             self.upsample_blocks[i] = layers.UpSampling1D(size=3)
#             self.conv_blocks[i] = layers.Conv1D(filters=num_channels, kernel_size=kernel_size[0], data_format='channels_last',
#                                                     padding=padding, kernel_initializer=init)
#             self.batchnorm_blocks[i] = layers.BatchNormalization(axis=-1, trainable=True)
#             self.ac_blocks[i] = layers.Activation('tanh') if decoder_type == 'linguistic' else layers.LeakyReLU(alpha=0.2)
#             self.drop_blocks[i] = layers.Dropout(rate=dropout_rate)
#         self.final_conv_block = layers.Conv1D(filters=num_channels, kernel_size=self.final_kernel_size, data_format='channels_last',
#                                                     padding='valid', kernel_initializer=init)
#         self.final_ac_block = layers.Softmax(axis=-1) if decoder_type == 'linguistic' else layers.LeakyReLU(alpha=0.2)
#         self.final_reshape_block = layers.Permute((2, 1))
    
#     def call(self, x):
#         import pdb
#         for i in range(self.num_levels-1):
#             x = self.upsample_blocks[i](x)
#             x = self.conv_blocks[i](x)
#             x = self.batchnorm_blocks[i](x)
#             x = self.ac_blocks[i](x)
#             x = self.drop_blocks[i](x) if self.training_state else x
#         pdb.set_trace()
#         len_padding = (self.output_len - (x.shape[1] - self.final_kernel_size) - 1)//2
#         x = tf.keras.layers.ZeroPadding1D(padding=(len_padding*2, 0))(x)
#         x = self.final_conv_block(x)
#         x = self.final_ac_block(x)
#         y = self.final_reshape_block(x)
#         return y

In [50]:
"""
sentenceEM.py
"""

class sentenceEM(tf.keras.Model):
    def __init__(self, encoder_args, linguistic_decoder_args, acoustic_decoder_args, input_shapes, seed, training_state):
        """
        1) encoder_args: 
        num_stacks, num_channels, kernel_size, dropout_rate, return_type, seed
        2) linguistic_decoder_args:
        decoder_type, num_channels, kernel_size, padding, dropout_rate, seed, training_state
        3) acoustic_decoder_args:
        decoder_type, num_channels, kernel_size, padding, dropout_rate, seed, training_state
        """
        super(sentenceEM, self).__init__()
        self.input_shapes = input_shapes
        self.input_reshape_block = layers.Permute((2, 1))
        self.encoder = TempConvnet(num_stacks=encoder_args['num_stacks'], num_channels=encoder_args['num_channels'],
                                        kernel_size=encoder_args['kernel_size'], dropout_rate=encoder_args['dropout_rate'],
                                        return_type=encoder_args['return_type'], seed=seed, training_state=training_state)
        self.linguistic_decoder = TempConvnet_Decoder(decoder_type=linguistic_decoder_args['decoder_type'], num_channels=linguistic_decoder_args['num_channels'], 
                                                      kernel_size=linguistic_decoder_args['kernel_size'], padding=linguistic_decoder_args['padding'],
                                                      dropout_rate=linguistic_decoder_args['dropout_rate'], seed=seed, training_state=training_state)
        self.acoustic_decoder = TempConvnet_Decoder(decoder_type=acoustic_decoder_args['decoder_type'], num_channels=acoustic_decoder_args['num_channels'], 
                                                    kernel_size=acoustic_decoder_args['kernel_size'], padding=acoustic_decoder_args['padding'],
                                                    dropout_rate=acoustic_decoder_args['dropout_rate'], seed=seed, training_state=training_state)
    
    def call(self, x):
        embedded_outputs = self.input_reshape_block(x)
        embedded_outputs = self.encoder(embedded_outputs)
        embedded_outputs = kb.expand_dims(embedded_outputs, axis=1)
        linguistic_outputs = self.linguistic_decoder(embedded_outputs)
        acoustic_outputs = self.acoustic_decoder(embedded_outputs)
        return linguistic_outputs, acoustic_outputs
    
    def encoder_call(self, x):
        embedded_outputs = self.input_reshape_block(x)
        embedded_outputs = self.encoder(embedded_outputs)
        return embedded_outputs
    
    def decoder_call(self, x):
        embedded_outputs = kb.expand_dims(x, axis=1)
        linguistic_outputs = self.linguistic_decoder(embedded_outputs)
        acoustic_outputs = self.acoustic_decoder(embedded_outputs)
        return linguistic_outputs, acoustic_outputs
    
    def build_seperate_graph(self):
        # Encoder graph
        x = tf.keras.Input(batch_shape=self.input_shapes)
        embedded_outputs = self.encoder_call(x)
        self.embedded_outputs_shape = embedded_outputs.shape
        self.encoder_model = tf.keras.Model(inputs=x, outputs=embedded_outputs)

        # Decoder graph
        embeddings = tf.keras.Input(batch_shape=self.embedded_outputs_shape)
        linguistic_outputs, acoustic_outputs = self.decoder_call(embeddings)
        self.decoder_model = tf.keras.Model(inputs=embeddings, outputs=[linguistic_outputs, acoustic_outputs])
    
    def build_total_graph(self):
        x = tf.keras.Input(batch_input_shape=self.input_shapes)
        y = self.encoder_model(x)
        l_y, a_y = self.decoder_model(y)
        self.model = tf.keras.Model(inputs=x, outputs=[l_y, a_y])

    # def model_visualize(self):
    #     self.model.summary()
    #     plot_model(self.model, to_file='/home/awesomericky/Lab_intern/Prof_Oh/Code/Speech2Pickup/image/sentenceEM.png')
    
    def encoder_model_compile(self, lr):
        optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
        self.encoder_model.compile(optimizer=optimizer)

    def model_compile(self, lr, loss_weights):
        optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
        self.model.compile(optimizer=optimizer, loss=[self.linguistic_loss_function, self.acoustic_loss_function],
                           loss_weights=[loss_weights['linguistic'], loss_weights['acoustic']],
                           metrics=[tf.keras.metrics.CategoricalAccuracy(), tf.keras.metrics.MeanSquaredError()])

    def model_train(self, X_train, Y_linguistic_train, Y_acoustic_train, batch_size, epochs, total_model_file_path, encoder_model_file_path):
        custom_callback = CustomCallback(total_model_file_path, encoder_model_file_path, self.model, self.encoder_model)
        self.model.fit(x=X_train, y=[Y_linguistic_train, Y_acoustic_train], batch_size=batch_size, epochs=epochs, verbose=1, callbacks=[custom_callback])
    
    def linguistic_loss_function(self, y_actual, y_predicted):
        epsil = 1e-5
        y_predicted = -kb.log(y_predicted + epsil)
        loss = layers.multiply([y_actual, y_predicted])
        loss = kb.sum(loss, axis=1)
        loss = kb.mean(loss, keepdims=False)
        return loss
    
    def acoustic_loss_function(self, y_actual, y_predicted):
        loss = layers.subtract([y_actual, y_predicted])
        loss = layers.multiply([loss, loss])
        loss = kb.mean(loss, keepdims=False)
        return loss

class CustomCallback(Callback):
    def __init__(self, total_model_file_path, encoder_model_file_path, total_model, encoder_model):
        self.total_model_file_path = total_model_file_path
        self.encoder_model_file_path = encoder_model_file_path
        self.total_model = total_model
        self.encoder_model = encoder_model
        self.monitor = 'model_1_categorical_accuracy'
        self.monitor_op = np.greater
        self.best = 0
        self.temp_loss = []
        self.temp_linguistic_loss = []
        self.temp_acoustic_loss = []
        self.temp_linguistic_accuracy = []
    
    def on_batch_end(self, batch, logs=None):
        self.temp_loss.append(logs.get('loss'))
        self.temp_linguistic_loss.append(logs.get('model_1_loss'))
        self.temp_acoustic_loss.append(logs.get('model_1_1_loss'))
        self.temp_linguistic_accuracy.append(logs.get('model_1_categorical_accuracy'))

    def on_epoch_end(self, epoch, logs=None):
        self.temp_loss.append(logs.get('loss'))
        self.temp_linguistic_loss.append(logs.get('model_1_loss'))
        self.temp_acoustic_loss.append(logs.get('model_1_1_loss'))
        self.temp_linguistic_accuracy.append(logs.get('model_1_categorical_accuracy'))
        # temp_val_loss = logs.get('val_loss')
        # temp_val_linguistic_accuracy = logs.get('val_model_1_categorical_accuracy')
        # temp_val_acoustic_loss = logs.get('val_model_1_1_mean_squared_error')

        # Logging data
        for iii in range(len(self.temp_loss)):
            wandb.log({'temp_loss': self.temp_loss[iii]})
            wandb.log({'temp_linguistic_loss': self.temp_linguistic_loss[iii]})
            wandb.log({'temp_acoustic_loss': self.temp_acoustic_loss[iii]})
            wandb.log({'temp_linguistic_accuracy': self.temp_linguistic_accuracy[iii]})
        # wandb.log({'temp_val_loss': temp_val_loss})
        # wandb.log({'temp_val_linguistic_accuracy': temp_val_linguistic_accuracy})
        # wandb.log({'temp_val_acoustic_loss': temp_val_acoustic_loss})
        self.temp_loss = []
        self.temp_linguistic_loss = []
        self.temp_acoustic_loss = []
        self.temp_linguistic_accuracy = []
        
        # Save model
        current = logs.get(self.monitor)
        if self.monitor_op(current, self.best):
            self.best = current
            self.total_model.save_weights(filepath=self.total_model_file_path, overwrite=True)
            self.encoder_model.save_weights(filepath=self.encoder_model_file_path, overwrite=True)

In [55]:
# Configure model

train_model_number = 10
batch_size = 1024; seed = 1; lr = 0.0001; epochs = 50; loss_weights={'linguistic': 100, 'acoustic': 0.005}
n_mels = 40
time_steps = 303
word_dic_size = 43
dropout_rate = 0.1
training_state = True
input_shapes = (None, n_mels, time_steps)
encoder_args = {'num_stacks': 3, 'num_channels':[n_mels for i in range(6)], 'kernel_size':3, 'dropout_rate': dropout_rate, 'return_type': 'end'}
linguistic_decoder_args = {'decoder_type': 'linguistic', 'num_channels': word_dic_size,
                           'kernel_size': [2, 2], 'padding': 'causal', 'dropout_rate': dropout_rate}
acoustic_decoder_args = {'decoder_type': 'acoustic', 'num_channels': n_mels,
                         'kernel_size': [2, 2], 'padding': 'causal', 'dropout_rate': dropout_rate}

with tf.device('/device:GPU:0'):
    sen_em_model = sentenceEM(encoder_args=encoder_args, linguistic_decoder_args=linguistic_decoder_args, acoustic_decoder_args=acoustic_decoder_args,
                                input_shapes=input_shapes, seed=seed, training_state=training_state)
    sen_em_model.build_seperate_graph()
    sen_em_model.build_total_graph()
    sen_em_model.model_compile(lr=lr, loss_weights=loss_weights)
# sen_em_model.model_visualize()

In [56]:
# Train model

wandb_config = {'batch_size':batch_size,
                'learning_rate': lr,
                'loss_weight (linguistic, acoustic)': (loss_weights['linguistic'], loss_weights['acoustic']),
                'channel_type': 'single',
                'n_mels': n_mels,
                'seed': seed,
                'date': '2020-12-05',
                'train_model_number': train_model_number,
                'memo': 'init std=0.1, Dropout_rate: 0.1'}
wandb_run = wandb.init(project='Speech2Pickup', name='sentenceEM', config=wandb_config)
total_model_file_path = '/content/drive/MyDrive/Speech2Pickup/sentenceEM_model/' + str(train_model_number) + '/total_model/sentenceEM_total_model'
encoder_model_file_path = '/content/drive/MyDrive/Speech2Pickup/sentenceEM_model/' + str(train_model_number) + '/encoder_model/sentenceEM_encoder_model'
model_configuration_file = '/content/drive/MyDrive/Speech2Pickup/sentenceEM_model/' + str(train_model_number) + '/model_config.txt'
relative_data_directory_path = '/content/drive/MyDrive/Speech2Pickup/data_v2.2_single_channel'

# Write model configuration in .txt file
with open(model_configuration_file, 'w') as f:
    f.write(str(wandb_config))

# # Load data
# print('Loading data...')
# data_file = listdir(relative_data_directory_path)[0]
# data = load_single_npz_data(relative_data_directory_path=relative_data_directory_path, file_name=data_file)
# acoustic_data = data['acoustic']
# linguistic_data = data['linguistic']

# Train model and save
print('Start training')
try:
    # Train
    with tf.device('/device:GPU:0'):
        sen_em_model.model_train(X_train=acoustic_data, Y_linguistic_train=linguistic_data, Y_acoustic_train=acoustic_data, batch_size=batch_size, epochs=epochs,
                                 total_model_file_path=total_model_file_path, encoder_model_file_path=encoder_model_file_path)
except KeyboardInterrupt:
    pass

Start training
Train on 40698 samples
Epoch 1/50
40698/40698 [==============================] - 48s 1ms/sample - loss: 390.3807 - model_1_loss: 3.7743 - model_1_1_loss: 2565.9370 - model_1_categorical_accuracy: 0.0074 - model_1_mean_squared_error: 0.0228 - model_1_1_categorical_accuracy: 0.0056 - model_1_1_mean_squared_error: 2565.8904
Epoch 2/50
40698/40698 [==============================] - 34s 841us/sample - loss: 350.9025 - model_1_loss: 3.3796 - model_1_1_loss: 2559.6655 - model_1_categorical_accuracy: 0.0053 - model_1_mean_squared_error: 0.0222 - model_1_1_categorical_accuracy: 0.0074 - model_1_1_mean_squared_error: 2559.5754
Epoch 3/50
15360/40698 [==========>...................] - ETA: 21s - loss: 319.3015 - model_1_loss: 3.0653 - model_1_1_loss: 2555.2476 - model_1_categorical_accuracy: 0.0062 - model_1_mean_squared_error: 0.0214 - model_1_1_categorical_accuracy: 0.0076 - model_1_1_mean_squared_error: 2555.2478

In [57]:
# Erase model from memory
# Finish wandb process

tf.keras.backend.clear_session()
del sen_em_model
wandb_run.finish()

temp_loss,350.90249
_step,327
_runtime,101
_timestamp,1607104234
temp_linguistic_loss,3.37956
temp_acoustic_loss,2559.66553
temp_linguistic_accuracy,0.0053


temp_loss,█▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▃
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_runtime,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁████████████████████
_timestamp,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁████████████████████
temp_linguistic_loss,████▇▇▇▇▇▇▇▆▆▆▆▆▆▆▆▆▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁
temp_acoustic_loss,█▄▅▄▅▅▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▂▂▂▂▁▂▂▂▂▂▂▂▂▂▁▂▁▁▁
temp_linguistic_accuracy,██▇▇▇▆▆▆▅▅▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁


In [ ]:
x = tf.convert_to_tensor(np.random.random((batch_size, 40, 303)), dtype=np.float32)   # batch, dim, seq_len
l_y, a_y = sen_em_model.model(x)
embed = sen_em_model.encoder_model(x)
print(kb.eval(embed).shape)

print('='*20)
print(kb.eval(l_y).shape)
print(kb.eval(a_y).shape)
# print(len(kb.eval(l_y)[0,:,0]))
print('='*20)
print(np.sum(kb.eval(l_y)[0,:,0]))
print(np.sum(kb.eval(l_y)[0,:,100]))

In [ ]:
# Load model (model compiling should be done first)

model_path = '/content/drive/MyDrive/Speech2Pickup/sentenceEM_model/' + str(train_model_number) + '/total_model/sentenceEM_total_model'
sen_em_model.model.load_weights(model_path)

In [ ]:
"""
utils.py
"""

def read_script_files(script_dir_path):
    script_files = [f for f in listdir(script_dir_path) if isfile(join(script_dir_path, f))]
    script_files.sort()
    return script_files

def read_script_file_data(script_dir_path, script_file):
    curr_file = open(join(script_dir_path, script_file), 'r')
    curr_file_lines = curr_file.readlines()
    for i in range(len(curr_file_lines)):
        words = curr_file_lines[i].split()[2: ]
        curr_file_lines[i] = ' '.join(words)
    return curr_file_lines

In [ ]:
"""
process_data.py
"""

def make_word_dictionary(relative_script_directory_path):
    # Check whole word candidates
    script_files = read_script_files(relative_script_directory_path)
    total_words = set()
    num = 0
    for script_file in script_files:
        num += 1
        print('Processing {}/{}'.format(num, len(script_files)))
        curr_file_lines = read_script_file_data(relative_script_directory_path, script_file)
        for curr_file_line in curr_file_lines:
            total_words.update(set(curr_file_line.split()))
    total_words = list(total_words)
    total_words.append("")  # blank
    total_words.append("OOV")  # Out of vocabulary
    total_words.sort()

    # One-hot encoding
    word_dictionary = dict()
    word_dictionary_size = len(total_words)
    for i in range(word_dictionary_size):
        one_hot = np.zeros(word_dictionary_size)
        one_hot[i] = 1
        word_dictionary[total_words[i]] = one_hot
    # print(word_dictionary)
    return word_dictionary, word_dictionary_size

In [ ]:
# Set word dictionary to check output of model later

relative_script_directory_path = './drive/MyDrive/Speech2Pickup/train_script'
word_dic, word_dic_size = make_word_dictionary(relative_script_directory_path)

In [ ]:
# Check output of loaded model

import matplotlib.pyplot as plt
import librosa.display

relative_data_directory_path = '/content/drive/MyDrive/Speech2Pickup/data_v2.2_single_channel'
file_name = 'senEM_preprocessed.npz'
n = 40123
# data = load_single_npz_data(relative_data_directory_path=relative_data_directory_path, file_name=file_name)
acoustic_train_batch = [data['acoustic'][n]]
linguistic_train_batch = [data['linguistic'][n]]
acoustic_train_batch = np.array(acoustic_train_batch, dtype=np.float32)
linguistic_train_batch = np.array(linguistic_train_batch, dtype=np.float32)
l_out, a_out = sen_em_model.model(acoustic_train_batch)

l_out = np.squeeze(kb.eval(l_out))
a_out = np.squeeze(kb.eval(a_out))
l_true = np.squeeze(linguistic_train_batch)
a_true = np.squeeze(acoustic_train_batch)

l_out = np.argmax(l_out, axis=0)
l_true = np.argmax(l_true, axis=0)

true_sentence = []
predicted_sentence = []
word_dic_keys = list(word_dic.keys())
n = 0
for i in l_true:
  if n==0:
    true_sentence.append(word_dic_keys[i])
  elif true_sentence[-1] != word_dic_keys[i]:
    true_sentence.append(word_dic_keys[i])
  n += 1
n = 0
for i in l_out:
  if n==0:
    predicted_sentence.append(word_dic_keys[i])
  elif predicted_sentence[-1] != word_dic_keys[i]:
    predicted_sentence.append(word_dic_keys[i])
  n += 1
true_sentence = ' '.join(true_sentence)
predicted_sentence = ' '.join(predicted_sentence)
print('True: {}'.format(true_sentence))
print('Predict: {}'.format(predicted_sentence))

fig, ax = plt.subplots(1)
ax.plot(l_out, label='prediction')
ax.plot(l_true, label='ground truth')
ax.set_title('Linguistic feature')
ax.set_xlabel('Time step')
ax.set_ylabel('Freq')
plt.legend()
plt.show()

fig, ax = plt.subplots(nrows=2, ncols=1, sharex=True, figsize=(25,15))
img = librosa.display.specshow(data=a_true, sr=8000, hop_length=256,  x_axis='time',
                            y_axis='mel', cmap='viridis', fmax=4000, ax=ax[0])
ax[0].set(title='Mel specrogram - ground truth')
ax[0].label_outer()

img = librosa.display.specshow(data=a_out, sr=8000, hop_length=256,  x_axis='time',
                            y_axis='mel', cmap='viridis', fmax=4000, ax=ax[1])
ax[1].set(title='Mel specrogram - prediction')
ax[1].label_outer()

In [ ]:
# Test and save for jointly training 'Speech2Pickup' (Debugging)

sen_em_model.model.save_weights(filepath='/content/drive/MyDrive/Speech2Pickup/sentenceEM_model/total_model/sentenceEM_total_model', overwrite=True)
sen_em_model.encoder_model.save_weights(filepath='/content/drive/MyDrive/Speech2Pickup/sentenceEM_model/encoder_model/sentenceEM_encoder_model', overwrite=True)
print('Model saving complete!')

Model saving complete!


In [ ]:
# Compare sorting time

import time
ts = time.time()
random.shuffle(data_files)
te = time.time()
print('String sort: {}'.format(te-ts))
print(data_files)
# ts = time.time()
# random.shuffle(indexs)
# te = time.time()
# print('Int sort: {}'.format(te-ts))

In [ ]:
# Test (Debugging)

data = load_single_npz_data(relative_data_directory_path='/content/drive/MyDrive/Speech2Pickup/data_v2.2', file_name='senEM_preprocessed_100.npz')
data1 = data['arr_0']
data1 = np.float32(data1)
data1 = kb.expand_dims(data1, axis=0)
with tf.device('/device:GPU:0'):
  l_y, a_y = sen_em_model(data1)
print(kb.eval(a_y)[0,:,180])
print('='*20)
print(kb.eval(l_y)[0,:,180])
print(len(kb.eval(l_y)[0,:,180]))
print('='*20)
print(np.sum(kb.eval(l_y)[0,:,180]))
print(np.sum(kb.eval(l_y)[0,:,300]))

In [13]:
pdb.enable()

In [ ]:
# Test (Debugging)
import matplotlib.pyplot as plt
import librosa.display

x = tf.convert_to_tensor(np.ones((1, 40, 303)), dtype=np.float32)   # batch, dim, seq_len
l_y, a_y = sen_em_model.model(x)
print(kb.eval(l_y).shape)
print(kb.eval(a_y).shape)
# mid = sen_em_model.encoder_model(x)
# print('='*20)
# print(kb.eval(l_y)[0,:,200])
# out = np.squeeze(kb.eval(l_y))
# subtract = out[:, :10] - np.fliplr(out[:, 293:])
# print(subtract)
# print(out[:, :10])
# print('='*20)
# print(out[:, 293:])
# plt.imshow(out[:, :10], aspect='auto')
# plt.show()
# plt.imshow(out[:, 293:], aspect='auto')
# print(len(kb.eval(l_y)[0,:,:]))
# print(np.sum(kb.eval(l_y)[0,:,0]))
# print(np.sum(kb.eval(l_y)[0,:,100]))

In [ ]:
# print(kb.eval(l_y)[0,:,234:238])
# print(kb.eval(l_y)[0,:,298:])

In [ ]:
x = tf.convert_to_tensor(np.ones((1, 40, 303)), dtype=np.float32)   # batch, dim, seq_len
embedding = sen_em_model.encoder_model(x)
print(kb.eval(embedding))